In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

# Display more rows and get rid of the margins
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Display mulitiple values from each cell
# https://stackoverflow.com/a/42476224
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Define paths and functions

In [2]:
abcd_table_path = 'tables/'
data_dir = Path('data')

In [72]:
def load_abcd_table(path):
    table = pd.read_csv(path, skiprows=[1], header=0, sep='\t')
    labels = pd.read_csv(path, nrows=1, header=0, sep='\t')
    labels = labels.T.reset_index().rename(columns={'index':'name', 0:'doc'})
    return table, labels

def append_abcd_table(path, table=None, label=None, on=None):
    if table is None:
        return load_abcd_table(path)
    if on is None:
        on = ['subjectkey', 'interview_date', 'visit', 'lmt_run']
    new_table, new_label = load_abcd_table(path)
    # Find drop common columns that aren't in the set we're merging on
    drop_cols = label.loc[label.doc.isin(new_label.doc) & ~label.name.isin(on),'name'].values
    new_table.drop(drop_cols, axis=1, inplace=True)
    new_label = new_label[~new_label.name.isin(drop_cols) & ~new_label.name.isin(on)]
    
    # Get table shapes for error checking
    ts = table.shape
    nts = new_table.shape
    ls = label.shape
    nls = new_label.shape
    
    # Merge the tables
    table = table.merge(new_table, how='outer',on=on)
    
    # Check for success
    assert table.shape[0] == ts[0] == nts[0]
    assert table.shape[1] == (ts[1] + nts[1] - len(on))
    
    # Merge labels
    label = pd.concat([label, new_label]).reset_index(drop=True)
    
    assert label.shape[0] == table.shape[1]
    
    return table, label

def load_task_melt_contrasts(paths, contrasts, task_name):
    task_t = None
    task_l = None
    for pp in paths:
        task_t, task_l = append_abcd_table(pp, table=task_t, label=task_l)
        
    # Rearrange columns
    new_col_order = pd.concat([task_l.name[~task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0].isin(contrasts)],
                               task_l.name[task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0].isin(contrasts)]]).values
    task_t = task_t.loc[:,new_col_order]
    task_l = task_l.set_index('name').T.loc[:,new_col_order].T.reset_index()
    
    # Manipulate labels to get more information about them
    if task_name == 'nb':
        task_l.loc[task_l.doc.str.split('for ').str[1].str.split(' in').str[0].isin(contrasts), 'contrast'] = task_l.doc.str.split('for ').str[1].str.split(' in').str[0]
        task_l.loc[pd.notnull(task_l.contrast),'roi_code'] = task_l.loc[pd.notnull(task_l.contrast),'name'].str.split('x').str[-1]
        task_l.loc[pd.notnull(task_l.contrast),'hemisphere'] = task_l.loc[pd.notnull(task_l.contrast),'roi_code'].str.split('g').str[0]
        task_l.loc[pd.notnull(task_l.contrast),'roi_num'] = task_l.loc[pd.notnull(task_l.contrast),'roi_code'].str.split('p').str[-1].astype(int)
        task_meta_cols = task_l.name[~task_l.doc.str.split('for ').str[1].str.split(' in').str[0].isin(contrasts)]
    else:
        task_l.loc[task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0].isin(contrasts), 'contrast'] = task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0]
        task_l.loc[pd.notnull(task_l.contrast),'roi_code'] = task_l.loc[pd.notnull(task_l.contrast),'name'].str.split('x').str[-1]
        task_l.loc[pd.notnull(task_l.contrast),'hemisphere'] = task_l.loc[pd.notnull(task_l.contrast),'roi_code'].str.split('g').str[0]
        task_l.loc[pd.notnull(task_l.contrast),'roi_num'] = task_l.loc[pd.notnull(task_l.contrast),'roi_code'].str.split('p').str[-1].astype(int)
        task_meta_cols = task_l.name[~task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0].isin(contrasts)]

    task_t_melt = []
    for contrast in contrasts:
        tmpdf = task_t.loc[:,task_meta_cols]
        tmpdf['contrast'] = contrast
        if task_name == 'nb':
            contrast_cols = task_l.name[task_l.doc.str.split('for ').str[1].str.split(' in').str[0] == contrast].values
        else:
            contrast_cols = task_l.name[task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0] == contrast].values
        contrast_df = task_t.loc[:,contrast_cols]
        contrast_df.columns = [task_l.loc[task_l.name==cc,'roi_code'].values[0] for cc in contrast_df.columns.values]
        task_t_melt.append(pd.concat([tmpdf,contrast_df], axis=1))
    task_t_melt = pd.concat(task_t_melt)
    task_t_melt['task'] = task_name
    return task_t_melt, task_l

# Prepare data
## Get sex, handedness, age and device data for balancing

In [18]:
# Get Handedness data
ehis_path = abcd_table_path + 'abcd_ehis01.txt'
ehis = pd.read_csv(ehis_path, skiprows=[1], header=0, sep='\t')
ehis_labels= pd.read_csv(ehis_path, nrows = 1, header = 0, sep='\t')
ehis_labels = ehis_labels.T.reset_index().rename(columns={'index':'name', 0:'doc'})

# Look through all the tables with serial number to pull device data for all subjects and sessions
device_info_tables = ['abcd_betnet01.txt',
                      'mri_rsi01.txt',
                      'mrirscor01.txt',
                      'pdti401.txt']

device_info_fields = ['subjectkey',
                      'interview_date',
                      'eventname',
                      'scanner_manufacturer_pd',
                      'scanner_type_pd', 
                      'deviceserialnumber']
device_df = []
for di in device_info_tables:
    tmp = pd.read_csv(abcd_table_path + di,
                      skiprows = [1],
                      header = 0,
                      sep='\t').loc[:,device_info_fields]
    device_df.append(tmp)
device_df = pd.concat(device_df)
device_df = (device_df
             .loc[pd.notnull(device_df.scanner_manufacturer_pd),:]
             .drop_duplicates(keep='first')
             .reset_index())

# Put the handedness, age, and sex information together with the device data
bal_ehis_fields = ['collection_id',
                   'dataset_id', 
                   'subjectkey',
                   'interview_age', 
                   'gender',
                   'ehi_y_ss_scoreb',
                   'eventname']
bal_device_fields = ['subjectkey',
                     'scanner_manufacturer_pd',
                     'scanner_type_pd', 
                     'deviceserialnumber',
                     'eventname']

for_balancing = (ehis.loc[:,bal_ehis_fields]
                     .merge(device_df.loc[:,bal_device_fields],
                            how='outer',
                            on=['subjectkey', 'eventname'],
                            indicator=True))

In [19]:
# See how well the merge worked
for_balancing.groupby("_merge").count()

# Keep the columns present in both and drop the merge column
for_balancing = for_balancing.query('_merge == "both"').drop('_merge', axis=1)

# Save out the csv in case we need it anywhere else:
# for_balancing.to_csv('abcd_for_balancing.csv', index=None)

,collection_id,dataset_id,subjectkey,interview_age,gender,ehi_y_ss_scoreb,eventname,scanner_manufacturer_pd,scanner_type_pd,deviceserialnumber
_merge,,,,,,,,,,
left_only,324,324,324,324,324,324,324,0,0,0
right_only,0,0,0,0,0,0,0,0,0,0
both,4200,4200,4200,4200,4200,4200,4200,4200,4200,4200


There are 324 participants who got an Edinburgh Handedness Inventory who weren't in any of the tables with the 'deviceserialnumber' field.

## Assemble rs-fMRI dataframe

In [27]:
# Between network table
bn_t, bn_l = load_abcd_table(abcd_table_path + 'abcd_betnet01.txt')
# Network to subcortical table
ns_t, ns_l = load_abcd_table(abcd_table_path + 'mrirscor01.txt')
# See which columns are in both tables
bn_l.loc[bn_l.doc.isin(ns_l.doc),:]

,name,doc
0,collection_id,collection_id
1,dataset_id,dataset_id
2,subjectkey,The NDAR Global Unique Identifier (GUID) for research subject
3,src_subject_id,Subject ID how it's defined in lab/project
4,interview_date,Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY
5,interview_age,Age in months at the time of the interview/test/sampling/imaging.
6,gender,Sex of the subject
7,eventname,The event name for which the data was collected
8,visit,Visit name
9,rsfm_tr,pulse repetition time in seconds


In [28]:
# We'll merge on subjectkey and interview_date, filter out the other columns from ns_t before merging
ns_t.drop(bn_l.loc[bn_l.doc.isin(ns_l.doc),'name'].drop([2,4]).values,
          axis=1,
          inplace=True)

con = bn_t.merge(ns_t,
                 how='outer',
                 on=['subjectkey','interview_date'])

con_l = pd.concat([bn_l, 
                   ns_l[~ns_l.name.isin(bn_l.loc[bn_l.doc.isin(ns_l.doc),'name'].values)]]).reset_index(drop=True)

In [29]:
# Merge in balancing variables
con = con.merge(for_balancing.loc[:,['subjectkey','eventname','ehi_y_ss_scoreb']], how='left',
          on=['subjectkey','eventname'])

con_l = con_l.append(pd.DataFrame([{'name':'ehi_y_ss_scoreb','doc':'Handedness score rating'}]))

# Get a list of Metric columns
con_meta_cols = ['collection_id', 'dataset_id', 'subjectkey',
            'src_subject_id', 'interview_date', 'interview_age',
            'gender', 'event_name', 'visit', 'rsfm_tr', 'eventname',
            'rsfm_nreps', 'rsfm_numtrs', 'pipeline_version',  'scanner_manufacturer_pd',
            'scanner_type_pd','deviceserialnumber', 'magnetic_field_strength',
            'procdate', 'collection_title', 'promoted_subjectkey',
            'study_cohort_name','ehi_y_ss_scoreb', 'qc_ok']
con_metric_cols = con_l.loc[~con_l.name.isin(con_meta_cols), 'name'].values


# pull out the roi and network names 

con_l['from_network'] = con_l.loc[con_l.name.isin(con_metric_cols),'doc'].str.split('between ').str[1].str.split('network').str[0]
con_l['to'] = con_l.loc[con_l.name.isin(con_metric_cols),'doc'].str.split('between ').str[1].str.split('network').str[1].str.split(' and').str[1].str.replace('ASEG ROI', '')
con_l['roi'] = con_l.loc[con_l.name.isin(con_metric_cols),'doc'].str.split('and ASEG ROI').str[1].str.strip()

con_l['from_network'].unique()
con_l.roi.unique()

array([nan, 'auditory ', 'cingulo-opercular ', 'cingulo-parietal ',
       'default ', 'dorsal attention ', 'fronto-parietal ', '"none" ',
       'retrosplenial temporal ', 'sensorimotor hand ',
       'sensorimotor mouth ', 'salience ', 'ventral attention ', 'visual '], dtype=object)

array([nan, 'left-cerebellum-cortex', 'left-thalamus-proper',
       'left-caudate', 'left-putamen', 'left-pallidum', 'brain-stem',
       'left-hippocampus', 'left-amygdala', 'left-accumbens-area',
       'left-ventraldc', 'right-cerebellum-cortex',
       'right-thalamus-proper', 'right-caudate', 'right-putamen',
       'right-pallidum', 'right-hippocampus', 'right-amygdala',
       'right-accumbens-area', 'right-ventraldc'], dtype=object)

## Assemble task based tables

### Tstats

In [31]:
# Load Monetary Incentive Delay data
mid_paths = ['tstgodp101','tstgodp201','tstgodp301', 'tstgodp401']
mid_paths = [abcd_table_path + pp + '.txt' for pp in mid_paths]

mid_contrasts = ['MID anticipation of reward versus neutral',
       'MID anticipation of loss versus neutral',
       'MID reward positive versus negative feeback',
       'MID loss positive versus negative feedback',
       'MID anticipation of large reward versus neutral',
       'MID anticipation of small reward versus neutral',
       'MID anticipation of large versus small reward',
       'MID anticipation of large loss versus neutral',
       'MID anticipation of small loss versus neutral',
       'MID anticipation of small versus large loss']

mid_t_melt, mid_l = load_task_melt_contrasts(mid_paths, mid_contrasts, 'mid')

# Load Stop Signal Task

sst_paths = ['ssttstatp101','ssttstatp201', 'ssttstatp301']
sst_paths = [abcd_table_path + pp + '.txt' for pp in sst_paths]

sst_contrasts = ['SST correct go versus fixation',
       'SST correct stop versus correct go',
       'SST incorrect stop versus correct go',
       'SST any stop versus correct go',
       'SST correct stop versus incorrect stop',
       'SST incorrect go versus correct go',
       'SST incorrect go versus incorrect stop']

sst_t_melt, sst_l = load_task_melt_contrasts(sst_paths, sst_contrasts, 'sst')


nb_paths = ['nbacktstatp101','nbacktstatp201', 'nbacktstatp301', 'nbacktstatp401']
nb_paths = [abcd_table_path + pp+'.txt' for pp in nb_paths]

nb_contrasts = ['nBack 0 back condition', 'nBack 2 back condition',
       'nBack place condition',
       'nBack emotion condition',
       'nBack 2 back versus 0 back contrast',
       'nBack face versus place contrast',
       'nBack emotion versus neutral face contrast',
       'nBack negative face versus positive face contrast',
       'nBack positive face versus neutral face contrast']

nb_t_melt, nb_l = load_task_melt_contrasts(nb_paths, nb_contrasts, 'nb')


In [32]:
tb_t = pd.concat([mid_t_melt, sst_t_melt, nb_t_melt])
tb_t = tb_t.merge(for_balancing.loc[:,['subjectkey','eventname','ehi_y_ss_scoreb','scanner_manufacturer_pd',
            'scanner_type_pd', 'deviceserialnumber']],
                  how='left',
                  on=['subjectkey','eventname'])

### Betas

In [74]:
# Load Monetary Incentive Delay data
mid_paths = ['gordonp101','gordonp201','gordonp301', 'gordonp401']
mid_paths = [abcd_table_path + pp + '.txt' for pp in mid_paths]

mid_contrasts = ['MID anticipation of reward versus neutral',
       'MID anticipation of loss versus neutral',
       'MID reward positive versus negative feeback',
       'MID loss positive versus negative feedback',
       'MID anticipation of large reward versus neutral',
       'MID anticipation of small reward versus neutral',
       'MID anticipation of large versus small reward',
       'MID anticipation of large loss versus neutral',
       'MID anticipation of small loss versus neutral',
       'MID anticipation of small versus large loss']

mid_b_melt, mid_l = load_task_melt_contrasts(mid_paths, mid_contrasts, 'mid')

# Load Stop Signal Task

sst_paths = ['mrisstp101','mrisstp201', 'mrisstp301']
sst_paths = [abcd_table_path + pp + '.txt' for pp in sst_paths]

sst_contrasts = ['SST correct go versus fixation',
       'SST correct stop versus correct go',
       'SST incorrect stop versus correct go',
       'SST any stop versus correct go',
       'SST correct stop versus incorrect stop',
       'SST incorrect go versus correct go',
       'SST incorrect go versus incorrect stop']

sst_b_melt, sst_l = load_task_melt_contrasts(sst_paths, sst_contrasts, 'sst')


nb_paths = ['nbackgordonp101','nbackgordonp201', 'nbackgordonp301', 'nbackgordonp401']
nb_paths = [abcd_table_path + pp+'.txt' for pp in nb_paths]

nb_contrasts = ['nBack 0 back condition', 'nBack 2 back condition',
       'nBack place condition',
       'nBack emotion condition',
       'nBack 2 back versus 0 back contrast',
       'nBack face versus place contrast',
       'nBack emotion versus neutral face contrast',
       'nBack negative face versus positive face contrast',
       'nBack positive face versus neutral face contrast']

nb_b_melt, nb_l = load_task_melt_contrasts(nb_paths, nb_contrasts, 'nb')

In [75]:
tb_b = pd.concat([mid_b_melt, sst_b_melt, nb_b_melt])
tb_b = tb_b.merge(for_balancing.loc[:,['subjectkey','eventname','ehi_y_ss_scoreb','scanner_manufacturer_pd',
            'scanner_type_pd', 'deviceserialnumber']],
                  how='left',
                  on=['subjectkey','eventname'])

# Filter out bad scans


In [76]:
qc_t, qc_l = load_abcd_table(abcd_table_path + 'mriqc01.txt')
qc_t = qc_t.rename(columns={'visit':'eventname'})

In [77]:
qc_cols = ['iqc_rsfmri_good_ser',
 'iqc_mid_good_ser',
 'iqc_sst_good_ser',
 'iqc_nback_good_ser']

In [78]:
scan_types = ['rsfmri','nback','mid','sst']
qc_col_pat = 'iqc_%s_good_ser'
for st in scan_types:
    col = qc_col_pat%st
    if st == 'rsfmri':
         qc_t[st+'_ok'] = (qc_t.loc[:,col]==4).astype(bool)
    else:
        qc_t[st+'_ok'] = (qc_t.loc[:,col]==2).astype(bool)
#qc_ok_cols = ['iqc_%s_1_qc_score'%st for st in scan_types]

In [79]:
(qc_t.loc[:, qc_cols]==2).sum()

iqc_rsfmri_good_ser     378
iqc_mid_good_ser       3299
iqc_sst_good_ser       3215
iqc_nback_good_ser     3151
dtype: int64

In [80]:
con = con.merge(qc_t.loc[:, ['subjectkey', 'eventname', 'rsfmri_ok']], how='left', on=['subjectkey','eventname']).head()

In [81]:
con = con.rename(columns={'rsfmri_ok':'qc_ok'})

In [82]:
con.query('qc_ok == 0')

,collection_id,dataset_id,subjectkey,src_subject_id,interview_date,interview_age,gender,eventname,visit,rsfm_tr,rsfm_nreps,rsfm_numtrs,rsfm_mean_motion,rsfm_max_motion,rsfm_mean_trans,rsfm_max_trans,rsfm_mean_rot,rsfm_max_rot,scanner_manufacturer_pd,scanner_type_pd,deviceserialnumber,magnetic_field_strength,pipeline_version,procdate,rsfm_auditory_to_auditory,rsfm_auditory_to_cingulooperc,rsfm_auditory_to_cingulopar,rsfm_auditory_to_default,rsfm_auditory_to_dorsalattn,rsfm_auditory_to_frontopariet,rsfm_auditory_to_none,rsfm_aud_to_retsplenialtem,rsfm_auditory_to_smhand,rsfm_auditory_to_smmouth,rsfm_auditory_to_salience,rsfm_auditory_to_ventralattn,rsfm_auditory_to_visual,rsfm_cingulooperc_to_auditory,rsfm_copc_to_copc,rsfm_copc_to_cinguloparietal,rsfm_copc_to_default,rsfm_copc_to_dorsalattn,rsfm_copc_to_frontoparietal,rsfm_cingulooperc_to_none,rsfm_copc_to_retsplenialtem,rsfm_cingulooperc_to_smhand,rsfm_cingulooperc_to_smmouth,rsfm_cingulooperc_to_salience,rsfm_copc_to_ventralattn,rsfm_cingulooperc_to_visual,rsfm_cpar_to_auditory,rsfm_cpar_to_cingulooperc,rsfm_cpar_to_cpar,rsfm_cpar_to_default,rsfm_cpar_to_dorsalattn,rsfm_cpar_to_frontoparietal,rsfm_cinguloparietal_to_none,rsfm_cinloparl_to_retsplen,rsfm_cinguloparietal_to_smhand,rsfm_cpar_to_smmouth,rsfm_cpar_to_salience,rsfm_cpar_to_ventralattn,rsfm_cinguloparietal_to_visual,rsfm_default_to_auditory,rsfm_default_to_cingulooperc,rsfm_default_to_cpar,rsfm_default_to_default,rsfm_default_to_dorsalattn,rsfm_default_to_frontoparietal,rsfm_default_to_none,rsfm_default_to_retsplenialtem,rsfm_default_to_smhand,rsfm_default_to_smmouth,rsfm_default_to_salience,rsfm_default_to_ventralattn,rsfm_default_to_visual,rsfm_dorsalattn_to_auditory,rsfm_dorsalattn_to_cperc,rsfm_dorsalattn_to_cpar,rsfm_dorsalattn_to_default,rsfm_dorsalattn_to_dorsalattn,rsfm_dorsalattn_to_fpar,rsfm_dorsalattn_to_none,rsfm_dorsalattn_to_retsplem,rsfm_dorsalattn_to_smhand,rsfm_dorsalattn_to_smmouth,rsfm_dorsalattn_to_salience,rsfm_dorsalattn_to_ventralattn,rsfm_dorsalattn_to_visual,rsfm_fp_to_auditory,rsfm_fp_to_cingulooperc,rsfm_fp_to_cinguloparietal,rsfm_fp_to_default,rsfm_fp_to_dorsalattn,rsfm_fp_to_fp,rsfm_frontoparietal_to_none,rsfm_fopar_to_retsplenialtem,rsfm_frontoparietal_to_smhand,rsfm_fpar_to_smmouth,rsfm_fpar_to_salience,rsfm_fpar_to_ventralattn,rsfm_frontoparietal_to_visual,rsfm_none_to_auditory,rsfm_none_to_cingulooperc,rsfm_none_to_cinguloparietal,rsfm_none_to_default,rsfm_none_to_dorsalattn,rsfm_none_to_frontoparietal,rsfm_none_to_none,rsfm_none_to_retrplenialtem,rsfm_none_to_smhand,rsfm_none_to_smmouth,rsfm_none_to_salience,rsfm_none_to_ventralattn,rsfm_none_to_visual,rsfm_rst_to_auditory,rsfm_rst_to_cingulooperc,rsfm_rst_to_cinguloparietal,rsfm_rst_to_default,rsfm_rst_to_dorsalattn,rsfm_rst_to_frontoparietal,rsfm_rst_to_none,rsfm_rst_to_rst,rsfm_rst_to_smhand,rsfm_rst_to_smmouth,rsfm_rst_to_salience,rsfm_rst_to_ventralattn,rsfm_rst_to_visual,rsfm_smhand_to_auditory,rsfm_smhand_to_cingulooperc,rsfm_smhand_to_cpar,rsfm_smhand_to_default,rsfm_smhand_to_dorsalattn,rsfm_smhand_to_fpar,rsfm_smhand_to_none,rsfm_smhand_to_rst,rsfm_smhand_to_smhand,rsfm_smhand_to_smmouth,rsfm_smhand_to_salience,rsfm_smhand_to_ventralattn,rsfm_smhand_to_visual,rsfm_smmouth_to_auditory,rsfm_smmouth_to_cingulooperc,rsfm_smmouth_to_cpar,rsfm_smmouth_to_default,rsfm_smmouth_to_dorsalattn,rsfm_smmouth_to_fpar,rsfm_smmouth_to_none,rsfm_smmouth_to_rst,rsfm_smmouth_to_smhand,rsfm_smmouth_to_smmouth,rsfm_smmouth_to_salience,rsfm_smmouth_to_ventralattn,rsfm_smmouth_to_visual,rsfm_salience_to_auditory,rsfm_salience_to_cingulooperc,rsfm_salience_to_cpar,rsfm_salience_to_default,rsfm_salience_to_dorsalattn,rsfm_salience_to_fpar,rsfm_salience_to_none,rsfm_salience_to_rst,rsfm_salience_to_smhand,rsfm_salience_to_smmouth,rsfm_salience_to_salience,rsfm_salience_to_ventralattn,rsfm_salience_to_visual,rsfm_ventralattn_to_auditory,rsfm_ven_to_cingulooperc,rsfm_ven_to_cinguloparietal,rsfm_ventralattn_to_default,rsfm_ven_to_dorsalattn,rsfm_ven_to

In [83]:
tb_qc = []
for st in scan_types:
    if st != 'rsfmri':
        tmpdf = qc_t.loc[:, ['subjectkey', 'eventname', st+'_ok']].rename(columns={st+'_ok':'qc_ok'})
        if st == 'nback':
            tmpdf['task'] = 'nb'
        else:
            tmpdf['task'] = st
        tb_qc.append(tmpdf)
tb_qc = pd.concat(tb_qc)

In [84]:
tb_qc.task.unique()
tb_t.task.unique()


array(['nb', 'mid', 'sst'], dtype=object)

array(['mid', 'sst', 'nb'], dtype=object)

In [85]:
tb_t.merge(tb_qc, how='outer', on=['subjectkey', 'eventname', 'task'], indicator=True).groupby('_merge').count()
tb_b.merge(tb_qc, how='outer', on=['subjectkey', 'eventname', 'task'], indicator=True).groupby('_merge').count()

,collection_id,collection_title,contrast,dataset_id,eventname,fmri_beta_gparc_mean_motion,fmri_beta_gparc_numtrs,fmri_beta_gparc_tr,gender,interview_age,interview_date,lhgp1,lhgp10,lhgp100,lhgp101,lhgp102,lhgp103,lhgp104,lhgp105,lhgp106,lhgp107,lhgp108,lhgp109,lhgp11,lhgp110,lhgp111,lhgp112,lhgp113,lhgp114,lhgp115,lhgp116,lhgp117,lhgp118,lhgp119,lhgp12,lhgp120,lhgp121,lhgp122,lhgp123,lhgp124,lhgp125,lhgp126,lhgp127,lhgp128,lhgp129,lhgp13,lhgp130,lhgp131,lhgp132,lhgp133,lhgp134,lhgp135,lhgp136,lhgp137,lhgp138,lhgp139,lhgp14,lhgp140,lhgp141,lhgp142,lhgp143,lhgp144,lhgp145,lhgp146,lhgp147,lhgp148,lhgp149,lhgp15,lhgp150,lhgp151,lhgp152,lhgp153,lhgp154,lhgp155,lhgp156,lhgp157,lhgp158,lhgp159,lhgp16,lhgp160,lhgp161,lhgp17,lhgp18,lhgp19,lhgp2,lhgp20,lhgp21,lhgp22,lhgp23,lhgp24,lhgp25,lhgp26,lhgp27,lhgp28,lhgp29,lhgp3,lhgp30,lhgp31,lhgp32,lhgp33,lhgp34,lhgp35,lhgp36,lhgp37,lhgp38,lhgp39,lhgp4,lhgp40,lhgp41,lhgp42,lhgp43,lhgp44,lhgp45,lhgp46,lhgp47,lhgp48,lhgp49,lhgp5,lhgp50,lhgp51,lhgp52,lhgp53,lhgp54,lhgp55,lhgp56,lhgp57,lhgp58,lhgp59,lhgp6,lhgp60,lhgp61,lhgp62,lhgp63,lhgp64,lhgp65,lhgp66,lhgp67,lhgp68,lhgp69,lhgp7,lhgp70,lhgp71,lhgp72,lhgp73,lhgp74,lhgp75,lhgp76,lhgp77,lhgp78,lhgp79,lhgp8,lhgp80,lhgp81,lhgp82,lhgp83,lhgp84,lhgp85,lhgp86,lhgp87,lhgp88,lhgp89,lhgp9,lhgp90,lhgp91,lhgp92,lhgp93,lhgp94,lhgp95,lhgp96,lhgp97,lhgp98,lhgp99,lmt_run,mid_beta_seg_dof,nback_place_tstat_ctxrhgp307,promoted_subjectkey,rhgp162,rhgp163,rhgp164,rhgp165,rhgp166,rhgp167,rhgp168,rhgp169,rhgp170,rhgp171,rhgp172,rhgp173,rhgp174,rhgp175,rhgp176,rhgp177,rhgp178,rhgp179,rhgp180,rhgp181,rhgp182,rhgp183,rhgp184,rhgp185,rhgp186,rhgp187,rhgp188,rhgp189,rhgp190,rhgp191,rhgp192,rhgp193,rhgp194,rhgp195,rhgp196,rhgp197,rhgp198,rhgp199,rhgp200,rhgp201,rhgp202,rhgp203,rhgp204,rhgp205,rhgp206,rhgp207,rhgp208,rhgp209,rhgp210,rhgp211,rhgp212,rhgp213,rhgp214,rhgp215,rhgp216,rhgp217,rhgp218,rhgp219,rhgp220,rhgp221,rhgp222,rhgp223,rhgp224,rhgp225,rhgp226,rhgp227,rhgp228,rhgp229,rhgp230,rhgp231,rhgp232,rhgp233,rhgp234,rhgp235,rhgp236,rhgp237,rhgp238,rhgp239,rhgp240,rhgp241,rhgp242,rhgp243,rhgp244,rhgp245,rhgp246,rhgp247,rhgp248,rhgp249,rhgp250,rhgp251,rhgp252,rhgp253,rhgp254,rhgp255,rhgp256,rhgp257,rhgp258,rhgp259,rhgp260,rhgp261,rhgp262,rhgp263,rhgp264,rhgp265,rhgp266,rhgp267,rhgp268,rhgp269,rhgp270,rhgp271,rhgp272,rhgp273,rhgp274,rhgp275,rhgp276,rhgp277,rhgp278,rhgp279,rhgp280,rhgp281,rhgp282,rhgp283,rhgp284,rhgp285,rhgp286,rhgp287,rhgp288,rhgp289,rhgp290,rhgp291,rhgp292,rhgp293,rhgp294,rhgp295,rhgp296,rhgp297,rhgp298,rhgp299,rhgp300,rhgp301,rhgp302,rhgp303,rhgp304,rhgp305,rhgp306,rhgp307,rhgp308,rhgp309,rhgp310,rhgp311,rhgp312,rhgp313,rhgp314,rhgp315,rhgp316,rhgp317,rhgp318,rhgp319,rhgp320,rhgp321,rhgp322,rhgp323,rhgp324,rhgp325,rhgp326,rhgp327,rhgp328,rhgp329,rhgp330,rhgp331,rhgp332,rhgp333,src_subject_id,study_cohort_name,subjectkey,task,visit,ehi_y_ss_scoreb,scanner_manufacturer_pd,scanner_type_pd,deviceserialnumber,qc_ok_x,qc_ok_y
_merge,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
left_only,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

,collection_id,collection_title,contrast,dataset_id,eventname,fmri_beta_gparc_mean_motion,fmri_beta_gparc_numtrs,fmri_beta_gparc_tr,gender,interview_age,interview_date,lhgp1,lhgp10,lhgp100,lhgp101,lhgp102,lhgp103,lhgp104,lhgp105,lhgp106,lhgp107,lhgp108,lhgp109,lhgp11,lhgp110,lhgp111,lhgp112,lhgp113,lhgp114,lhgp115,lhgp116,lhgp117,lhgp118,lhgp119,lhgp12,lhgp120,lhgp121,lhgp122,lhgp123,lhgp124,lhgp125,lhgp126,lhgp127,lhgp128,lhgp129,lhgp13,lhgp130,lhgp131,lhgp132,lhgp133,lhgp134,lhgp135,lhgp136,lhgp137,lhgp138,lhgp139,lhgp14,lhgp140,lhgp141,lhgp142,lhgp143,lhgp144,lhgp145,lhgp146,lhgp147,lhgp148,lhgp149,lhgp15,lhgp150,lhgp151,lhgp152,lhgp153,lhgp154,lhgp155,lhgp156,lhgp157,lhgp158,lhgp159,lhgp16,lhgp160,lhgp161,lhgp17,lhgp18,lhgp19,lhgp2,lhgp20,lhgp21,lhgp22,lhgp23,lhgp24,lhgp25,lhgp26,lhgp27,lhgp28,lhgp29,lhgp3,lhgp30,lhgp31,lhgp32,lhgp33,lhgp34,lhgp35,lhgp36,lhgp37,lhgp38,lhgp39,lhgp4,lhgp40,lhgp41,lhgp42,lhgp43,lhgp44,lhgp45,lhgp46,lhgp47,lhgp48,lhgp49,lhgp5,lhgp50,lhgp51,lhgp52,lhgp53,lhgp54,lhgp55,lhgp56,lhgp57,lhgp58,lhgp59,lhgp6,lhgp60,lhgp61,lhgp62,lhgp63,lhgp64,lhgp65,lhgp66,lhgp67,lhgp68,lhgp69,lhgp7,lhgp70,lhgp71,lhgp72,lhgp73,lhgp74,lhgp75,lhgp76,lhgp77,lhgp78,lhgp79,lhgp8,lhgp80,lhgp81,lhgp82,lhgp83,lhgp84,lhgp85,lhgp86,lhgp87,lhgp88,lhgp89,lhgp9,lhgp90,lhgp91,lhgp92,lhgp93,lhgp94,lhgp95,lhgp96,lhgp97,lhgp98,lhgp99,lmt_run,mid_beta_seg_dof,promoted_subjectkey,rhgp162,rhgp163,rhgp164,rhgp165,rhgp166,rhgp167,rhgp168,rhgp169,rhgp170,rhgp171,rhgp172,rhgp173,rhgp174,rhgp175,rhgp176,rhgp177,rhgp178,rhgp179,rhgp180,rhgp181,rhgp182,rhgp183,rhgp184,rhgp185,rhgp186,rhgp187,rhgp188,rhgp189,rhgp190,rhgp191,rhgp192,rhgp193,rhgp194,rhgp195,rhgp196,rhgp197,rhgp198,rhgp199,rhgp200,rhgp201,rhgp202,rhgp203,rhgp204,rhgp205,rhgp206,rhgp207,rhgp208,rhgp209,rhgp210,rhgp211,rhgp212,rhgp213,rhgp214,rhgp215,rhgp216,rhgp217,rhgp218,rhgp219,rhgp220,rhgp221,rhgp222,rhgp223,rhgp224,rhgp225,rhgp226,rhgp227,rhgp228,rhgp229,rhgp230,rhgp231,rhgp232,rhgp233,rhgp234,rhgp235,rhgp236,rhgp237,rhgp238,rhgp239,rhgp240,rhgp241,rhgp242,rhgp243,rhgp244,rhgp245,rhgp246,rhgp247,rhgp248,rhgp249,rhgp250,rhgp251,rhgp252,rhgp253,rhgp254,rhgp255,rhgp256,rhgp257,rhgp258,rhgp259,rhgp260,rhgp261,rhgp262,rhgp263,rhgp264,rhgp265,rhgp266,rhgp267,rhgp268,rhgp269,rhgp270,rhgp271,rhgp272,rhgp273,rhgp274,rhgp275,rhgp276,rhgp277,rhgp278,rhgp279,rhgp280,rhgp281,rhgp282,rhgp283,rhgp284,rhgp285,rhgp286,rhgp287,rhgp288,rhgp289,rhgp290,rhgp291,rhgp292,rhgp293,rhgp294,rhgp295,rhgp296,rhgp297,rhgp298,rhgp299,rhgp300,rhgp301,rhgp302,rhgp303,rhgp304,rhgp305,rhgp306,rhgp307,rhgp308,rhgp309,rhgp310,rhgp311,rhgp312,rhgp313,rhgp314,rhgp315,rhgp316,rhgp317,rhgp318,rhgp319,rhgp320,rhgp321,rhgp322,rhgp323,rhgp324,rhgp325,rhgp326,rhgp327,rhgp328,rhgp329,rhgp330,rhgp331,rhgp332,rhgp333,src_subject_id,study_cohort_name,subjectkey,task,visit,ehi_y_ss_scoreb,scanner_manufacturer_pd,scanner_type_pd,deviceserialnumber,qc_ok
_merge,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
left_only,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
right_o

In [86]:
tb_t = tb_t.merge(tb_qc, how='left', on=['subjectkey', 'eventname', 'task'])
tb_b = tb_b.merge(tb_qc, how='left', on=['subjectkey', 'eventname', 'task'])

In [87]:
con.to_pickle(data_dir/'con.pkz')
tb_t.to_pickle(data_dir/'task_based_tstats.pkz')
tb_b.to_pickle(data_dir/'task_based_betas.pkz')